## Generate large frames for testing

In [1]:
## NOTE: all units are kN and m

FD = {'storey_heights': [6.5] + [5.5]*20 + [7.0],   # m
      'bay_widths': [10.5,10,10,10,10,10.5],        # m
      'frame_spacing':8,                            # m, used only for load calculation
      'specified_loads':{'live':2.4,'dead':4.0,'snow':2.5,'wind':2.5},  # kPa
      'load_combinations':{'Case-2a':{'dead':1.25,'live':1.5,'snow':0.5},
                           'Case-2b':{'dead':1.25,'live':1.5,'wind':0.4},
                           'Case-3a':{'dead':1.25,'snow':1.5,'live':0.5},
                           'Case-3b':{'dead':1.25,'snow':1.5,'wind':0.4},
                           'Case-4a':{'dead':1.25,'wind':1.4,'live':0.5},
                           'Case-4b':{'dead':1.25,'wind':1.4,'snow':0.5},
                          },
      'load_combo':'Case-2b',
      'braced_bays':[0,2,3,5],
      'support_fixity': ['fx,fy,mz']*3 + ['fx,fy']*2 + ['fx,fy,mz']*2,
      'beam_size': 'W1000x222',
      'column_size': 'W360x216',
      }

SaveData = True
ShowResults = False

In [2]:
FD

{'bay_widths': [10.5, 10, 10, 10, 10, 10.5],
 'beam_size': 'W1000x222',
 'braced_bays': [0, 2, 3, 5],
 'column_size': 'W360x216',
 'frame_spacing': 8,
 'load_combinations': {'Case-2a': {'dead': 1.25, 'live': 1.5, 'snow': 0.5},
  'Case-2b': {'dead': 1.25, 'live': 1.5, 'wind': 0.4},
  'Case-3a': {'dead': 1.25, 'live': 0.5, 'snow': 1.5},
  'Case-3b': {'dead': 1.25, 'snow': 1.5, 'wind': 0.4},
  'Case-4a': {'dead': 1.25, 'live': 0.5, 'wind': 1.4},
  'Case-4b': {'dead': 1.25, 'snow': 0.5, 'wind': 1.4}},
 'load_combo': 'Case-2b',
 'specified_loads': {'dead': 4.0, 'live': 2.4, 'snow': 2.5, 'wind': 2.5},
 'storey_heights': [6.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  7.0],
 'support_fixity': ['fx,fy,mz',
  'fx,fy,mz',
  'fx,fy,mz',
  'fx,fy',
  'fx,fy',
  'fx,fy,mz',
  'fx,fy,mz']}

In [3]:
from salib import import_notebooks
import_notebooks()
from Frame2D import Frame2D
from Tables import Table
import numpy as np
import pandas as pd

In [6]:
def genframe(fd):
    h = fd['storey_heights']
    w = fd['bay_widths']
    s = fd['frame_spacing']
    nnodes = (len(h)+1)*(len(w)+1)
    
    # names of column stacks and floor levels
    bayline = [chr(ord('A')+i) for i in range(len(w)+1)]
    floorlev = [str(i) for i in range(len(h)+1)]
    
    # generate the nodes
    nodelist = []
    nidgrid = np.ndarray((len(h)+1,len(w)+1),dtype=np.object)
    for i in range(len(h)+1):
        y = sum(h[:i])*1000.
        for j in range(len(w)+1):
            x = sum(w[:j])*1000.
            nid = bayline[j]+floorlev[i]
            nodelist.append((nid,x,y))
            nidgrid[i,j] = nid
    nodes = pd.DataFrame(nodelist,columns=['NODEID','X','Y'])
    
    # generate the supports
    assert len(fd['support_fixity'])==nidgrid.shape[1]
    supplist = []
    for j,s in enumerate(fd['support_fixity']):
        nid = nidgrid[0,j]
        fix = s.strip().upper().split(',')
        if len(fix) < 3:
            fix += [np.nan] * (3-len(fix))
        supplist.append([nid,]+fix)
    supports = pd.DataFrame(supplist,columns=['NODEID','C0','C1','C2'])
    
    # generate columns
    columns = []
    for i in range(nidgrid.shape[0]-1):
        for j in range(nidgrid.shape[1]):
            nidj = nidgrid[i,j]
            nidk = nidgrid[i+1,j]
            mid = 'C' + nidj + nidk
            columns.append((mid,nidj,nidk))
    
    # generate beams
    beams = []
    roofbeams = []
    pinnedbeams = []
    for i in range(1,nidgrid.shape[0]):
        beamlist = beams if i < nidgrid.shape[0]-1 else roofbeams
        for j in range(nidgrid.shape[1]-1):
            nidj = nidgrid[i,j]
            nidk = nidgrid[i,j+1]
            mid = 'B' + nidj + nidk
            beamlist.append((mid,nidj,nidk))
            if j not in fd['braced_bays']:
                pinnedbeams.append(mid)
    members = pd.DataFrame(columns+beams+roofbeams,columns=['MEMBERID','NODEJ','NODEK'])
    
    # generate releases
    rellist = []
    for mid in pinnedbeams:
        rellist.append((mid,'MZJ'))
        rellist.append((mid,'MZK'))
    releases = pd.DataFrame(rellist,columns=['MEMBERID','RELEASE'])
    
    # generate properties
    proplist = []
    size = fd['column_size']
    for mid,j,k in columns:
        proplist.append((mid,size,np.nan,np.nan))
        size = np.nan
    size = fd['beam_size']
    for mid,j,k in beams+roofbeams:
        proplist.append((mid,size,np.nan,np.nan))
        size = np.nan
    properties = pd.DataFrame(proplist,columns=['MEMBERID','SIZE','IX','A'])
    
    # generate node loads (wind from left)
    nloadlist = []
    L = fd['specified_loads']   # area loads
    for i in range(1,nidgrid.shape[0]+1):
        H = (sum(h[:i+1])-sum(h[:i-1]))/2.
        FL = H*fd['frame_spacing']*L['wind']
        if FL != 0.:
            nloadlist.append(('wind',nidgrid[i,0],'FX',FL*1000.))
    node_loads = pd.DataFrame(nloadlist,columns=['LOAD','NODEID','DIRN','F'])
    
    # generate member loads
    mloadlist = []
    UDL = -L['dead']*fd['frame_spacing']
    mloadlist += [('dead',mid,'UDL',UDL,np.nan,np.nan,np.nan,np.nan) for mid,nj,nk in beams]
    mloadlist += [('dead',mid,'UDL',UDL,np.nan,np.nan,np.nan,np.nan) for mid,nj,nk in roofbeams]
    UDL = -L['live']*fd['frame_spacing']
    mloadlist += [('live',mid,'UDL',UDL,np.nan,np.nan,np.nan,np.nan) for mid,nj,nk in beams]
    UDL = -L['snow']*fd['frame_spacing']
    mloadlist += [('snow',mid,'UDL',UDL,np.nan,np.nan,np.nan,np.nan) for mid,nj,nk in roofbeams]
    member_loads = pd.DataFrame(mloadlist,columns='LOAD,MEMBERID,TYPE,W1,W2,A,B,C'.split(','))
    
    # generate load combinations
    lclist = []
    for case,loads in fd['load_combinations'].items():
        for load,factor in loads.items():
            lclist.append((case,load,factor))
    load_combinations = pd.DataFrame(lclist,columns=['CASE','LOAD','FACTOR'])
    
    Table.set_source(None,ds_type='data')
    Table.set_data('nodes',nodes)
    Table.set_data('supports',supports)
    Table.set_data('members',members)
    Table.set_data('releases',releases)
    Table.set_data('properties',properties)
    Table.set_data('node_loads',node_loads)
    Table.set_data('member_loads',member_loads)
    Table.set_data('load_combinations',load_combinations)
    
    frame = Frame2D()
    frame.input_all()
    
    return frame

In [7]:
%time f = genframe(FD)

CPU times: user 396 ms, sys: 28 ms, total: 424 ms
Wall time: 405 ms


In [8]:
if SaveData:
    NS = len(FD['storey_heights'])
    NB = len(FD['bay_widths'])
    name = 'l{}x{}'.format(NS,NB)
    f.write_all(name,mkdir=True)

In [9]:
%time rs = f.solve(FD['load_combo'])
if ShowResults:
    f.print_input()
    f.print_results(rs)

CPU times: user 116 ms, sys: 4 ms, total: 120 ms
Wall time: 115 ms


In [ ]:
#%matplotlib notebook
#f.plot()